In [ ]:
#import neccessary modules 
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn.preprocessing import Normalizer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.utils.validation import check_is_fitted



In [ ]:
#load csv files
train = pd.read_csv('C:/Users/Yusuf/Desktop/Portfolio Project/walmart/train.csv')
test = pd.read_csv('C:/Users/Yusuf/Desktop/Portfolio Project/walmart/test.csv')
features = pd.read_csv('C:/Users/Yusuf/Desktop/Portfolio Project/walmart/features.csv')
store = pd.read_csv('C:/Users/Yusuf/Desktop/Portfolio Project/walmart/stores.csv')
submission = pd.read_csv('C:/Users/Yusuf/Desktop/Portfolio Project/walmart/sampleSubmission.csv')

In [ ]:
#Data inspection and manipulation
train.head()

In [ ]:
train.tail()

In [ ]:
train.info()

In [ ]:
train.shape

In [ ]:
train.describe()

In [ ]:
test.head()

In [ ]:
features.head()

In [ ]:
features.tail()

In [ ]:
features.shape

In [ ]:
store.head()

In [ ]:
store.shape

In [ ]:
#Data Cleaning and Manipulation
#check for missing values
train.isnull().sum()

In [ ]:
store.isnull().sum()

In [ ]:
features.isnull().sum()

In [ ]:
print(train.shape)
features.shape

In [ ]:
df = train.merge(features).merge(store)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
#converting date column to date datatype
df['Date'] = pd.to_datetime(df['Date'])



In [ ]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Week'] = df['Date'].dt.week

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df

Explanatory Data Analysis(EDA)

In [ ]:
#No of stores available
print("No of stores available", df.Store.nunique())
print(df.Store.unique())

In [ ]:
#No of dept available
print("No of Dept. available", df.Dept.nunique())
print("List of available dept.", (df.Dept.unique()))


In [ ]:
#Type of stores available
print("No of type of stores available:", df.Type.nunique())
print("List of available stores", (df.Type.unique()))

In [ ]:
df.info()

In [ ]:
df.Size.unique()

In [ ]:
df.head()

In [ ]:
numerical = df[['Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Size']]
categorical = ['Store', 'Dept', 'Date', 'IsHoliday', 'Type']

In [ ]:
df.tail()

Which stores has the highest number of frequency 

In [ ]:
fig = plt.figure(figsize=(14, 6))
df["Store"].value_counts().plot(kind='bar')
plt.xlabel('Store')
plt.ylabel('Frequency')
plt.title("No of Stores")

Which type is the most commonnest store type

In [ ]:
fig = plt.figure(figsize=(18, 6))
df["Type"].value_counts().plot(kind='bar')
plt.xlabel('Type')
plt.ylabel('No of Types')  

Insights

A is the most common store type

In [ ]:
fig = plt.figure(figsize=(18, 6))
df.groupby('Store')['Weekly_Sales'].mean().plot(kind='bar')

#which store and type has the highest sales
What is the average size of each store and type

In [ ]:
fig = plt.figure(figsize=(14, 6))
df.groupby('Store')['Size'].mean().sort_values(ascending=False).plot(kind='bar')

In [ ]:
fig = plt.figure(figsize=(18, 6))
df.groupby('Type')['Size'].mean().plot(kind='bar')
df.groupby('Type')['Size'].mean()

Highest sales per store

In [ ]:
fig = plt.figure(figsize=(18, 6))
df.groupby('Store')['Weekly_Sales'].sum().sort_values(ascending=False).plot(kind='bar');

In [ ]:
fig = plt.figure(figsize=(18, 6))
df.groupby('Type')['Weekly_Sales'].sum().sort_values(ascending=False).plot(kind='pie')

In [ ]:
#fig = plt.figure(figsize=(14,6))
#plt.bar(df['Day'], df['Weekly_Sales'])

plt.figure(figsize=(20,9))
sns.lineplot(data = df, x=df['Day'], y=df['Weekly_Sales'])

#plt.plot(xpoints, ypoints)
plt.show()

In [ ]:
#plt.figure(figsize=(20,9))
#plt.bar( x=df['Year'], height=df['Weekly_Sales'])

In [ ]:
plt.figure(figsize=(20,9))
sns.lineplot(data = df, x=df['Week'], y=df['Weekly_Sales'])

In [ ]:
plt.figure(figsize=(20,9))
sns.lineplot(data = df, x=df['Year'], y=df['Weekly_Sales'])

In [ ]:
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Size']
df[numeric_features + ['Weekly_Sales']].describe()

In [ ]:
df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# This ensures plots are displayed inline in the Jupyter notebook
%matplotlib inline

# Get the Target column
Target = df['Weekly_Sales']


# Create a figure for 2 subplots (2 rows, 1 column)
fig, ax = plt.subplots(2, 1, figsize = (9,12))

# Plot the histogram   
ax[0].hist(Target, bins=100)
ax[0].set_ylabel('Frequency')

# Add lines for the mean, median, and mode
ax[0].axvline(Target.mean(), color='magenta', linestyle='dashed', linewidth=2)
ax[0].axvline(Target.median(), color='cyan', linestyle='dashed', linewidth=2)

# Plot the boxplot   
ax[1].boxplot(Target, vert=False)
ax[1].set_xlabel('Weekly_Sales')

# Add a title to the Figure
fig.suptitle('Weekly_Sales Distribution')

# Show the figure
fig.show()


In [ ]:
df = train.merge(features).merge(store)

In [ ]:
df.head()

In [ ]:
#remove the top and bottom 10%
low, high = df['Weekly_Sales'].quantile([0.1, 0.9])
Weekly_Sales = df['Weekly_Sales'].between(low, high)

In [ ]:
df = df[Weekly_Sales]

In [ ]:
df.info()

remove columns that are more than 50% null

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'], axis=1, inplace = True)

In [ ]:
df.head()

In [ ]:
numerical = ['Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Size']
categorical = ['Store', 'Dept', 'Date', 'IsHoliday', 'Type']

In [ ]:
#drop high and low cardinality categorical features.

In [ ]:
df[categorical].nunique()

In [ ]:
#df.drop(['IsHoliday', 'Type'], axis=1, inplace=True)

In [ ]:
df.head()

check for correlation

In [ ]:
df.dtypes

In [ ]:
corr = df.select_dtypes(include =['float64', 'int64']).drop("Weekly_Sales", axis=1).corr()
#sns.heatmap(corr)
#corr = df[numerical].corr()
sns.heatmap(corr)

In [ ]:
corr

Split data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
X = df.drop(['Weekly_Sales', 'Date'], axis=1)
y = df['Weekly_Sales']

In [ ]:
X_train, X_test,  y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
X_train

Build Model

In [ ]:
#Baseline model

y_mean = y_train.mean()
y_baseline_pred = [y_mean] * len(y_train)

In [ ]:
mean_absolute_error(y_train, y_baseline_pred)

In [ ]:
print("Mean apt price:", y_train.mean())

print("Baseline MAE:", mean_absolute_error(y_train, y_baseline_pred))

In [ ]:



#X_train['Type'] = pd.get_dummies(df, columns = ['Type'])
#X_test['Type'] = pd.get_dummies(df, columns = ['Type'])
#print(one_hot_encoded_data)


In [ ]:
X_train

In [ ]:
model = make_pipeline(
    OneHotEncoder(),
    Normalizer(),
    LinearRegression()
)

model.fit(X_train, y_train)

In [ ]:
X_train.head()

In [ ]:
predictions = pd.Series(model.predict(X_test))

In [ ]:
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)

rmse = np.sqrt(mse)
print("RMSE:", rmse)

r2 = r2_score(y_test, predictions)
print("R2_SCORE:", r2)


# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Weekly_Sales Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

Try Lasso Regression

In [ ]:
model = make_pipeline(
    OneHotEncoder(),
    Normalizer(),
    Lasso()
)

# Fit a lasso model on the training set
model.fit(X_train, y_train)

In [ ]:
from sklearn.linear_model import Lasso


# Evaluate the model using the test data
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Weekly_Sales Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

Ridge regression

In [ ]:
model = make_pipeline(
    OneHotEncoder(),
    Normalizer(),
    Ridge()
)

# Fit a lasso model on the training set
model.fit(X_train, y_train)

In [ ]:
from sklearn.linear_model import Ridge

# Evaluate the model using the test data
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Weekly_Sales Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
#from sklearn.tree import export_text

# Train the model

model = make_pipeline(
    OneHotEncoder(),
    Normalizer(),
    DecisionTreeRegressor()
)


# Fit a lasso model on the training set
model.fit(X_train, y_train)


# Visualize the model tree
#tree = export_text(model)
#print(tree)

#Evaluate the model using the test data
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE: {:.2f}".format(rmse))
r2_score = r2_score(y_test, predictions)
print("R2_SCORE: {:2f}".format(r2_score))


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Train the model
model = RandomForestRegressor().fit(X_train, y_train)
print (model, "\n")

# Evaluate the model using the test data
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Weekly_Sales Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

In [ ]:
# Train the model
from sklearn.ensemble import GradientBoostingRegressor

# Fit a GradientBoostingRegressor model on the training set
model = GradientBoostingRegressor().fit(X_train, y_train)
print (model, "\n")

# Evaluate the model using the test data
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Weekly_Sales Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()